In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TABELA_ORIGEM = "v_credit.silver.tb_motivo"
TABELA_DESTINO = "v_credit.gold.dm_motivo"

In [0]:
df_silver = spark.read.table(TABELA_ORIGEM)

df_gold = df_silver.select(
    F.col("cd_motivo"),
    F.col("ds_motivo"),
    F.col("ds_categoria"),
    F.col("ds_criticidade"),
    
    F.when(F.upper(F.col("ds_criticidade")).like("%BAIXA%"), "Alto Potencial Automação")
    .when(F.upper(F.col("ds_criticidade")).like("%MEDIA%"), "Médio Potencial")
    .otherwise("Baixo (Requer Humano)").alias("ds_potencial_automacao")
)

# Merge
DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"), "t.cd_motivo = s.cd_motivo"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

print(f"✅ Dimensão Motivo carregada!")